### ***Your Name Here***

# Prelab 7 - Exploratory Data Analysis Tools

In [ ]:
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Part 1 - Multi-Panel and Multi-Dimensional Plots

So far in this class, we have dealt with a single figure, containing a single set of axes, and representing no more than 2 variables. However, it is often desirable to work with:
1) multiple plots at once, either in separate figures, or inside a single figure. Tufte sometimes calls simple multiple panel plots that facilitate comparisons between them "small multiples". This can be very effective for exploring and interrogating a new dataset. For example, you might make scatterplots of every numerical variable in the table vs. every other one to begin looking for relationships. 
or
2) more than two variables at once, so that the interrelationships between many variables can be more easily seen. 

Matplotlib makes it possible for us to do all of these things relatively easily, but requires some new syntax. Let's start with some basic terminology and concepts. The highest level object that a user of pyplot usually deals with is a ***figure***. One can think of a figure as representing a single plotting window, or, if we're writing output to files, a single file.

At the next level down, most things that we can plot inside a figure will go into axes. Axes are exactly what they sound like: a pair of x and y axes, which are characterized by having some range, as well as auxiliary data like tick marks, axis labels, titles, legends, etc. A figure can contain multiple axes, in which case multiple things will be drawn inside the same plotting window. Axes also contain information about their position within a figure. Figures and axes are both examples of graphics **containers**: they are things into which graphical elements can be placed.

At the next level down are things like lines, filled polygons (filled regions between lines, for example), text, etc. These are known as graphics ***primitives***. Collectively, primitives and containers are known as ***Artists***. They are the basic graphical elements out of which plots are built. Commands like plot, fill_between, etc., produce primitives and attach them to containers.

One figure and one axis are active at any given time. The active figure and axis are the ones into which the lines or other graphics primitives created by commands like plot will be placed. So if you use commands like plt.xlim, plt.ylabel, plt.legend without specifying an axis, it will not spit out an error, but will also set those quantities only for the currently active plot, not for all subplots within a figure. It is best practice to manually specify where (which axes) graphical elements should go, as in the example below. 

Even simple 2D plots can also depict more than two variables at a time. For example colors, plotting symbols, point sizes, and transparency values could all be used to represent different variables.

Finally, another Python package that will come in very handy in the rest of this course, which you've seen in a few other activities but not yet been explicitly introduced to, is the numpy random module. This allows you to generate random numbers and make random draws from statistical distributions of various functional forms (normal, exponential, poisson, etc.). As with our previous use of the scipy.stats package, statistical distributions need some number of input parameters to define their shape (for example the mean and standard deviation for the normal distribution), so you have to read up on each distribution to use these commands properly. 

<div class=hw>
    
### Exercise 1 - Multipanel, Multidimensional, Randomization Exploration
---------------
    
The cell below gives a demonstration of a multiple panel plot with various random number functions, plus third and fourth variables. Your task is to:
a) Copy the code into a new cell so that you can manipulate it and still have a record of the original. Read docstrings, modify variables, keywords, etc. until you're sure you know what each is doing. Once you're satisfied, write a comment above each line of the original cell describing what it's doing. 
b) For each of the three plots, write a sentence or two describing how random numbers and additional dimensions were used and give an example of when such a plot might be useful. 
c) In a new cell, add a fourth subplot to the figure in a 2x2 grid (*hint: the syntax for the axes will look like a matrix, namely ((ax1, ax2),(ax3,ax4))*. Your new plot should implement at least three data dimensions (e.g. x,y and a color) and at least one random number function that is not already used in the example. For a full list of random functions, see [this page](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html)

In [ ]:
#add comments to this cell for part (a), but DON'T ALTER THE CODE ITSELF HERE
#copy the code into the cell below so that you can manipulate it and figure out what's happening
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,3), sharey=True)
f.suptitle('Three plots')
#subplot 1
npoints=50
x = np.random.rand(npoints)
y=np.random.rand(npoints)
colors = np.random.rand(npoints)
im1 = ax1.scatter(x,y, c=colors)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
f.colorbar(im1,ax=ax1, label="z")
ax1.set_ylim(0,1)

#subplot 2
npoints2=100
loc=0.5
scale=0.15
x2=np.random.normal(loc,scale,npoints2)
y2=np.random.normal(loc,scale,npoints2)
colors2 = np.random.rand(npoints2)
sizes=np.random.normal(loc,scale*10,npoints2)*50
im2 = ax2.scatter(x2,y2,s=sizes,c=colors2,alpha=0.5,cmap='magma')
ax2.set_xlim(0,1)
ax2.set_xlabel("x")
f.colorbar(im2,ax=ax2, label="z")

#subplot 3
npoints3=100
loc2=[0.5,0.5]
scales=[0.2,0.2]
corrcoef=0.7
norm=scales[0]*scales[1]
cov_matrix=[[scales[0]**2,corrcoef*norm],[corrcoef*norm,scales[1]**2]]
x3,y3 = np.random.multivariate_normal(loc2,cov_matrix,npoints3).T
im3 = ax3.scatter(x3,y3,label='pop1')
loc3=[0.3,0.4]
scales2=[0.2,0.5]
corrcoef2=0.9
norm2=scales2[0]*scales2[1]
cov_matrix2=[[scales2[0]**2,corrcoef2*norm2],[corrcoef2*norm2,scales2[1]**2]]
x4,y4 = np.random.multivariate_normal(loc3,cov_matrix2,npoints3).T
im4 = ax3.scatter(x4,y4, label='pop2')
ax3.legend()
ax3.set_xlim(0,1)
ax3.set_xlabel("x")

In [ ]:
#copy cell above to here and manipulate at will until you know what everything does

In [ ]:
#new code with fourth dimension

## Part 2 - Histogram Binning Exercises
One important idea that we haven't talked too much about so far is the idea of "binning". This idea is applied in many scenarios in science, but we will consider it here for the case of histograms. Choosing bins for histograms is something of an art, and so its useful to know what to look for and what to avoid. We will use the QuaRCS dataset for this investigation so that we can compare ordinal and continuous variables. 

In [ ]:
quarcs_data=pd.read_csv('QuaRCS_Pre_forAST200_anonymized.csv', encoding="ISO-8859-1")
mask = np.where(quarcs_data == 999)
quarcs_data = quarcs_data.replace(999,np.nan) 

First, let's consider an ordinal (ordered, but discrete values) variable - score on the QuaRCS assessment. There are 25 possible values, so let's assign 25 bins.

In [ ]:
hist1 = plt.hist(quarcs_data["PRE_SCORE"],bins=25, edgecolor='k') 

<div class=hw>
    
### Exercise 2.1 - Downsampling Data
---------------
    
One thing we might like to do when making a histogram is take a coarser or finer look at the distribution. Let's start with a coarser look, which corresponds to downsampling or decreasing the number of bins.  Try changing the number of bins to 12 and then 6 and then 3. Place the three plots side-by-side in a single multipanel figure, then answer the following questions in the space below. 
    
a) What are the advantages and disadvantages of downsampling? 
b) What features of the distribution are dependent on sampling and which are preserved? 

***Your Answer Here***

In [ ]:
#your code for multipanel histogram plot here

<div class=hw>
    
### Exercise 2.2 - Upsampling Data
---------------
Another thing you can do is upsample your data, or increasing the number of bins. Try increasing the number of bins to 50 and then to 100, then make another three panel plot. Describe how this is similar or different from downsampling in the space below. 

***Your Answer Here***

In [ ]:
#your code for multipanel histogram plot here

<div class=hw>
    
### Exercise 2.3 - Continuous vs. Ordinal Variables
---------------
    
Look at the same phenomena (upsampling and downsampling) for a continuous variable by modifying the number of bins in the cell below. What considerations are the same as when you were studying the ordinal variable and which are different? What might the advantages and disadvantages of upsampling and downsampling be in this case? How do you know when you've gone too far either way?

***Your Answer Here***

In [ ]:
hist2 = plt.hist(quarcs_data["CF_Mean"],bins=10, edgecolor='k') 

## Binning Practicalities

For all of the plots we made so far, we allowed python to define the boundaries of the bins, but this "automatic binning" can be very problematic, as several of your previous plots will reveal if you look at them closely. For example, if we add ticks showing where the integer values of score lie on the score histogram with 25 bins, you might see something funny about them when comparing the ticks to the bins. 

In [ ]:
#auto binning (max - min)/nbins
plt.figure(figsize=(10,5))
hist3 = plt.hist(quarcs_data["PRE_SCORE"],bins=25, edgecolor='k') 
ticks=np.arange(0,26,1)
tk = plt.xticks(ticks)

### Histogram Tip 1 - when setting number of bins, make sure you understand where boundaries between them end up (e.g. by carefully examining graph)

Here we've naively specified 25 bins since there are 25 possible points. Since the minimum score is 1 for this dataset (it didn't have to be - 0 is actually the minimum possible score) and the maximum is 25, each bin has a width of (max-min)/nbins = (25 - 1)/25 or 24/25 = 0.96.

This causes some weirdness at the upper end of the plot, which you can see when you compare it to the same plot with bins separated by 1 point (24 bins) instead of 0.96 points. Now the ticks line up nicely with the boundaries of the bars, but something is also strange at the upper end of the plot. 

<div class=hw>
    
### Exercise 2.4 
---------------
Examine the plot below and then explain what happened to the final bin under the two scenarios shown here. 

***Your Answer Here***

In [ ]:
#side by side
plt.subplots(2,1, figsize=(10,5))
plt.subplot(211)
plt.hist(quarcs_data["PRE_SCORE"],bins=25,edgecolor='k') 
plt.title("bins = 25, spacing = 0.96")
tks = plt.xticks(ticks)
plt.subplot(212)
plt.hist(quarcs_data["PRE_SCORE"],bins=24,edgecolor='k') 
plt.title("bins = 24, spacing = 1")
ticks=np.arange(0,26,1)
tks = plt.xticks(ticks)
plt.tight_layout()

### Are bin boundaries exclusive or inclusive?

We might assume bin boundaries right at the integer values make the most sense BUT if you think about it, that is also confusing. Is a score with that value in the bin on the right or the left? Much more sense would be bins that are symmetric around the integer scores. These you must hard code, since the automatic binning will always start and end at the minimum and maximum values of the variable and will not go beyond them (so that the minimum value appears in the middle of a bin rather than at its edge)

In [ ]:
#better = user-defined bins
bin_def = np.arange(-0.5,26.5,1)
print(bin_def)

In [ ]:
plt.subplots(2,1, figsize=(10,5))
plt.subplot(211)
plt.hist(quarcs_data["PRE_SCORE"],bins=24,edgecolor='k')
plt.xlim(-1,26)
ticks=np.arange(0,26,1)
#ticks=[0,8,12]
plt.xticks(ticks)
plt.subplot(212)
plt.hist(quarcs_data["PRE_SCORE"],bins=bin_def,edgecolor='k')
plt.xlim(-1,26)
ticks=np.arange(0,26,1)
tks=plt.xticks(ticks)


This shows us that although the bin size (1 point) makes more sense when we specify bins=24, a bin defined between 24 and 25 will only include score=24, and not score =25, so the last set of scores gets left off. 

### Other problems with autobinning

Check out the histogram below. These are the same data, but now there are three distinct peaks. What happened?

In [ ]:
hist=plt.hist(quarcs_data["PRE_SCORE"],bins=20,edgecolor='k')

naively choosing some number of bins is also problematic because one bin may include more than one of the discrete allowed values, as here. The bins around 7, 13 and 19 include TWO possible scores (7/8, 13/14 and 19/20, and all other bins include only 1). This also happens in the case of 10 bins. 

In [ ]:
hist = plt.hist(quarcs_data["PRE_SCORE"],bins=10, edgecolor='k')

### So what if we want to downsample effectively?

Again, define bins that make sense by hand, where each includes the same integer number of possible scores (of course, this gets complicated when the number of possibilities/n bins is not itself an integer value. For example, binning total score by 2 leaves one of the 25 possible scores an "odd man out" since 25/2 is not an integer. In this case, we'll define the lowest bin as a score of 0 or 1, since 0 was technically allowed

In [ ]:
bin_def2=np.arange(0,28,2)-0.5 
print(bin_def2)
plt.hist(quarcs_data["PRE_SCORE"],bins=bin_def2,edgecolor='k')
plt.xticks(bin_def2+1)
plt.xlim(-0.5,25.5)

In [ ]:
#bin by five
#another trick = offset upper bound by a tiny amount so that it includes the upper score in the bin (5,10,15,20,25)
bin_def3=[0,5.001,10.001,15.001,20.001,25.001]
ticks=[2.5,7.5,12.5,17.5,22.5]

plt.hist(quarcs_data["PRE_SCORE"],bins=bin_def3, edgecolor='k')
plt.xticks(ticks)
plt.xlim(0,25)

For continuous variables, we don't have to be quite as careful, but there is a limit in that not all continuous variables are equally continuous. Many "continuous" variables were at some point calculated from discrete variables, which means there's a limitation to how many values they can take on. You can see this when you get high enough in bins for the average confidence rankings 


In [ ]:
hist = plt.hist(quarcs_data["CF_Mean"],bins=500,edgecolor='k')

<div class=hw>
    
### Exercise 2.5 - Summary
---------------
    
Based on what you learned in this exercise, summarize the main things to consider in creating histogram bins below. What are the clues when you are oversampling or undersampling? What general principles can you follow in making histograms in the future? What tests/sanity checks should you run whenever making a histogram?

*** Your Answer Here***

### Side Note: Binning in 2-Dimensions

As an aside, note that you can also bin in 2 dimensions, which has the effect of "smoothing" a 2D array (like an image)

In [ ]:
from astropy.io import fits

In [ ]:
def rebin( a, newshape ):
        '''Rebin an array to an arbitrary new shape.
        '''
        assert len(a.shape) == len(newshape)

        slices = [ slice(0,old, float(old)/new) for old,new in zip(a.shape,newshape) ]
        coordinates = np.mgrid[slices]
        indices = coordinates.astype('i')   #choose the biggest smaller integer index
        return a[tuple(indices)]

In [ ]:
def rebin_factor( a, newshape ):
        '''Rebin an array to a new shape.
        newshape must be a factor of a.shape.
        '''
        assert len(a.shape) == len(newshape)
        assert not sometrue(mod( a.shape, newshape ))

        slices = [ slice(None,None, old/new) for old,new in zip(a.shape,newshape) ]
        return a[slices]

In [ ]:
def rebin_simple(a, newshape):
        assert len(a.shape) == len(newshape)
        assert not sometrue(mod( a.shape, newshape ))

In [ ]:
im = fits.getdata('DSS_M51_15arcminsq.fits')

In [ ]:
plt.imshow(im, cmap='gray')
print(im.shape)

In [ ]:
im_rebin=rebin(im, (13.2,13.2))

In [ ]:
plt.imshow(im_rebin, cmap='gray')

<div class=hw>

### Exercise 3 - Final Project Database Investigation
---------------

For the last prelabs of the semester, some portion of the assignment will be preparation for your final project. This week, your job is to import a dataset that you think you might want to work with (from the list below or, by prior approval, any other dataset that you might be interested in working with) into python as pandas dataframes.  

Then:
1) Describe the nature of the dataset. What information does it contain and about what kind of astronomical objects? Who is it intended for? How were the data collected? How many objects/measurements does the database contain?

2) Provide some summary statistics and histograms (or box plots where appropriate if you're feeling fancy) for 3-5 interesting columns in the table. 

Note that your final project investigation needs to involve tabular data that can be imported into Pandas. This means that images, time series data, spectra, etc. are off limits (not because they aren't interesting - only because they are complex and outside of the scope of this class).

Most of the databases below are massive. In some cases, you may be able to import the entire archive and pare it down after importing it, but in many cases you will want to use the tools provided on the websites to select some interesting subset of the data. 

#### Suggested Databases

1) [The Gaia Archive](https://gea.esac.esa.int/archive/)     
2) [The IPAC Infrared Science Archive](https://irsa.ipac.caltech.edu/frontpage/)   
3) [Kepler and K2 Data Archives](https://keplerscience.arc.nasa.gov/data-products.html)    
4) [The Sloan Digital Sky Survey Catalog](https://www.sdss.org/dr12/data_access/)  
5) [NASA Extragalactic Database](https://ned.ipac.caltech.edu/byparams)    
6) [JPL Solar System Dynamics Databse](https://ssd.jpl.nasa.gov/?phys_data)   

In [ ]:
## import statements

In [ ]:
## visualizations and statistics

*** Explanations of datasets ***

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../custom.css", "r").read()
    return HTML(styles)
css_styling()